# CHAPTER 10 | Introduction to Articial Neural Networks with Keras

## The Perceptron

In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

In [2]:
iris = load_iris()

X = iris.data[:, (2, 3)]
y = (iris.target == 0).astype(np.int) # Is Iris Setosa ?

In [3]:
per_clf = Perceptron(max_iter=1000, tol=1e-3)
per_clf.fit(X, y)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [4]:
y_pred = per_clf.predict([[2, 0.5]])
y_pred # Should be 1, aka true

array([0])

## Implementing MLPs with Keras

In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
print(tf.__version__)
print(keras.__version__)

2.0.0
2.2.4-tf


### Building an image classifier user the sequential API

In [7]:
#Loading the dataset

fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [8]:
print(X_train_full.shape)
print(X_train_full.dtype)

(60000, 28, 28)
uint8


In [9]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [10]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [11]:
class_names[y_train[0]]

'Coat'

In [12]:
# Creating the Model using the sequential API

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation="relu"))
model.add(keras.layers.Dense(100, activation="relu"))
model.add(keras.layers.Dense(10, activation="softmax"))

InternalError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.

In [0]:
model.summary()

In [0]:
model.layers

In [0]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

### Training and Evaluating the Model

In [0]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

In [0]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [0]:
model.evaluate(X_test, y_test)

### Using the Model to Make Predictions

In [0]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

In [0]:
y_pred = model.predict_classes(X_new)
y_pred

In [0]:
np.array(class_names)[y_pred]

In [0]:
y_new = y_test[:3]
y_new

In [0]:
#p. 303 | Building a Regression MLP Using the Sequential API